In [1]:
# ONE‑CELL notebook: Lex section search ➜ Azure OpenAI RAG
# -------------------------------------------------------
# If you haven’t already run it in this kernel:
# %pip install --quiet "openai>=1.25.0" requests python-dotenv

import os, json, requests
from dotenv import load_dotenv
from openai import AzureOpenAI

# ---------- 0. config --------------------------------------------------------
load_dotenv()                                    # expects AZURE_OPENAI_ENDPOINT / KEY / DEPLOYMENT
LEX   = "http://localhost:8000"                  # Lex backend
MODEL = os.getenv("AZURE_OPENAI_DEPLOYMENT") or "gpt-4o"

oai = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key        = os.getenv("AZURE_OPENAI_KEY"),
    api_version    = "2025-04-01-preview",
)

# ---------- 1. retriever -----------------------------------------------------
def search_sections(query: str, k: int = 3):
    """POST to /legislation/section/search and return list of hits"""
    r = requests.post(
        f"{LEX}/legislation/section/search",
        json={"query": query, "size": k},
        timeout=30,
    )
    r.raise_for_status()
    return r.json()          # Lex already returns a list[dict]

# ---------- 2. RAG function --------------------------------------------------
def ask(question: str, k: int = 3) -> str:
    hits = search_sections(question, k)
    if not hits:
        return "I couldn’t find any passages."

    # Build context string using the real keys ('title', 'number', 'text', 'legislation_year')
    ctx = "\n\n".join(
        f"{h['title']} s.{h.get('number','?')} ({h['legislation_year']}): {h['text']}"
        for h in hits
    )

    messages = [
        {"role":"system",
         "content":"You are a UK public‑law assistant. Base answers strictly on the extracts provided."},
        {"role":"user",
         "content":f"Question: {question}\n\nExtracts:\n{ctx}"},
    ]

    resp = oai.chat.completions.create(model=MODEL, messages=messages)
    return resp.choices[0].message.content

# ---------- 3. demo ----------------------------------------------------------
print( ask("What duties does the Climate Change Act impose on the Secretary of State?") )

Based on the provided extracts, the Climate Change Act imposes the following duties on the Secretary of State:

### 1. **Meeting Environmental Targets (s.5)**
The Secretary of State has a duty to ensure that:
- Targets set under section 1 are met.
- The PM 2.5 air quality target set under section 2 is met.
- The species abundance target set under section 3 is met.

---

### 2. **Reporting on Environmental Targets (s.6)**
The Secretary of State must:
- Specify a reporting date for any target set under sections 1, 2, or 3.
- On or before that reporting date, lay before Parliament and publish a statement including:
  - Confirmation that the target has been met.
  - Notification that the target has not been met.
  - Notification that it is not yet possible to determine whether the target has been met, along with the reasons and intended steps to resolve this.
- If the target has not been met:
  - Lay before Parliament and publish a report within 12 months explaining:
    - Why the target h